Setup

In [ ]:
# this step reloads machine in order to prevent any error
%load_ext autoreload
%autoreload 2

In [1]:
# import statements
import requests

# ----------- Science Jubilee -------------
from science_jubilee import Machine as Jub
from science_jubilee.tools import HTTPSyringe as syringe
from science_jubilee.tools import Pipette
import time

import numpy as np
import pandas as pd

In [ ]:
# defines the jubilee to call
jubilee = Jub.Machine(address='192.168.1.2', simulated = False) 

In [ ]:
# this loads the well plate for the destination for samples (sample_holder_name, position)
samples = jubilee.load_labware('fisherbrand_96_wellplate_360ul.json', 2)
# this line of code defines the boundary conditions of the well plate requires 3 points
upper_left = (25.7,179.9)
upper_right = (124.9, 179.9)
lower_right = (124.9, 116.4)
samples.manual_offset([upper_left,upper_right,lower_right])
samples.load_manualOffset()

In [ ]:
# this is same as lines above just for stock solution
stocks = jubilee.load_labware('20mlscintillation_12_wellplate_18000ul.json', 3)
upper_left = (167.4,176.8)
upper_right = (258.0, 176.8)
lower_right = (258.0, 119.8)
stocks.manual_offset([upper_left,upper_right,lower_right])
stocks.load_manualOffset()

In [ ]:
# same as above just for pipette tip rack
tiprack = jubilee.load_labware('opentrons_96_tiprack_300ul.json', 0)
upper_left = (27.0,82.7)
upper_right = (125.6,82.7)
lower_right = (125.8,19.4)
tiprack.manual_offset([upper_left,upper_right,lower_right])
tiprack.load_manualOffset()

In [ ]:
# defines position of trash can for tips
trash = jubilee.load_labware('agilent_1_reservoir_290ml.json', 1)

In [ ]:
deck.safe_z

In [ ]:
# this code block defines all of our stocks
water_stock = stocks[0].bottom(+5)
red_stock = stocks[2].bottom(+5)
blue_stock = stocks[1].bottom(+5)
# include however many we need, remember to include however many 

Experiment Funcitons

In [ ]:
def transfer_red(v, sample):
    '''
    This function will take a volume to add, and from which stock to use

    Input - v (volume)(float)
          - stock (int) -position of the desired stock
          - sample (int) -position of the current sample
    Output - void, purpose of function    
    '''

    P300.transfer(vol=v, source_well=stocks[1] ,destination_well= samples[sample] , blowout=none, new_tip='once')
    return void

In [ ]:
def transfer_blue(v, sample):
    '''
    This function will take a volume to add, and from which stock to use

    Input - v (volume)(float)
          - stock (int)

    Output - void, purpose of function    
    '''

    P300.transfer(vol=v, source_well=stocks[2] ,destination_well=samples[sample] , blowout=none, new_tip='once')
    return void

In [ ]:
def transfer_water(v,sample):
    '''
    This function will take a volume to add, and from which stock to use

    Input - v (volume)(float)
          - sample (int)
    Output - void, purpose of function    
    '''
    # source well is always the same
    P300.transfer(vol=v, source_well=stocks[0] ,destination_well= samples[sample], blowout=none, new_tip='once')
    return void

In [ ]:
def mixer(sample_location):
    '''
    this function will mix a chosen sample
    '''
    P300.mix(500,3,location=sample_location)
    return void

Data for experiment

In [ ]:
#data = pd.read_csv('./data/for_igi.csv')

In [ ]:
# this is first time step, all the others can be done by increasing the index
# the dt1 is a list of [vol_a, vol_b, vol_h20, stock_a, stock_b]
#dt1 = data.loc[0]

In [ ]:
# for sample 1
#print(dt1)
#print(dt1.iloc[2])
vol_a = [1000,600,200]
vol_b = [0,200,400]
vol_water= [0,200,400]

In [ ]:
# for sample 1 dt1
transfer_red(vol_a[0], sample[0])
transfer_blue(vol_b[0], sample[0])
transfer_water(vol_water[0],sample[0])
mixer(sample[0])

In [ ]:
# want to see if it will work for a loop
for i in range2:
    transfer_red(vol_a[i+1], sample[0])
    transfer_blue(vol_b[i+1], sample[0])
    transfer_water(vol_water[i+1],sample[0])
    mixer(sample[0])